In [1]:
!pip install dash
!pip install dash_bootstrap_components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 4.8 MB/s eta 0:00:00


# Importation des librarry

In [2]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import base64
import nest_asyncio
import plotly.express as px
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import base64
import nest_asyncio
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import io


# Scaping web

**site de ansm**

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# DataFrame pour stocker les résultats
df = pd.DataFrame(columns=['Title', 'Author', 'Date', 'Number of Responses', 'Content'])

def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    post_content_tag = soup.find('div', class_='bbWrapper')
    post_content = post_content_tag.text.strip() if post_content_tag else "No content found"
    return post_content

def scrape_forum_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Les threads
    threads = soup.find_all('div', class_='structItem--thread')

    for thread in threads:
        title_tag = thread.find('div', class_='structItem-title')
        title = title_tag.text.strip()
        author = thread.find('a', class_='username').text.strip()
        date = thread.find('time')['datetime']

        # Nombre de réponses
        response_count_tag = thread.find('dl', class_='pairs pairs--justified structItem-minor')
        if response_count_tag:
            response_counts = response_count_tag.find_all('dd')
            if len(response_counts) > 1:
                num_responses = response_counts[1].text.strip()
            else:
                num_responses = '0'
        else:
            num_responses = '0'

        # Ontention complet du post
        post_url = 'https://forum.alzheimers.org.uk' + title_tag.find('a')['href']
        content = scrape_post_content(post_url)

        # Les données au DataFrame
        df.loc[len(df)] = [title, author, date, num_responses, content]

# Les 30 premières pages (600 postes)
base_url = 'https://forum.alzheimers.org.uk/forums/i-have-dementia.56/page-{}.html'
for page_num in range(1, 31):
    url = base_url.format(page_num)
    print(f'Scraping page {page_num}...')
    scrape_forum_page(url)
    time.sleep(1)

# Fichier CSV
df.to_csv('forum_posts_30_pages.csv', index=False)

print(df.head())


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
                                               Title    Author  \
0               Your advice - Appointing an attorney   TamsinT   
1  Webinars delivered by people living with dementia  HarrietD   
2                                     Hints and Tips     Pejic   
3  The dementia guide: for people dealing with a ...  HarrietD   
4                    I don’t know how to handle this     Nobo2   

            

**Forum alzheimer's Society**

In [4]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import urllib3

urllib3.disable_warnings()

url_list = [
    'https://data.ansm.sante.fr/specialite/62650485',
    'https://data.ansm.sante.fr/specialite/68673252',
    'https://data.ansm.sante.fr/specialite/66072114'
]

All_data_med = []
x = 0

# DataFrames pour stocker les données
df_medications = pd.DataFrame(columns=[
    'Nom_medicament', 'Estimation',
    'Hommes (%)',
    'Femmes (%)',
    '20-59 (%)',
    '60+ (%)',
    'Période_min', 'Période_max',
    'Total_declarations',
    'Hommes_decl (%)',
    'Femmes_decl (%)'
])
df_effects = pd.DataFrame(columns=[
    'Nom_medicament', 'Pathologie', 'Valeur'
])

for url in url_list:
    html_content = requests.get(url, verify=False).text
    soup = BeautifulSoup(html_content, "lxml")

    # Extraction des informations de base
    Nom_medicament = soup.find('div', attrs={'class': 'text-xl lg:text-2xl font-medium mb-4'})
    Estimation = soup.find('h3', attrs={'class': 'text-2xl text-primary'})

    if Nom_medicament:
        Nom_medicament = Nom_medicament.string
    else:
        Nom_medicament = "Unknown"

    if Estimation:
        Estimation = Estimation.string
    else:
        Estimation = "Unknown"

    script = soup.find('script', attrs={'id': "__NEXT_DATA__"}).string
    script2 = json.loads(script)

    Nom_medicament = script2["props"]["pageProps"]["cis"]['name']

    pat_trait_hom = script2["props"]["pageProps"]["cis"]["substances"][0]["repartitionPerGender"]["male"]
    pat_trait_fem = script2["props"]["pageProps"]["cis"]["substances"][0]["repartitionPerGender"]["female"]

    age_data = script2["props"]["pageProps"]["cis"]["substances"][0]['repartitionPerAge']
    age_20_59 = age_data[0] if len(age_data) > 0 else {'range': '20-59', 'valuePercent': None}
    age_60_p = age_data[1] if len(age_data) > 1 else {'range': '60+', 'valuePercent': None}

    periode_effects = script2["props"]["pageProps"]["cis"]["substances"][0]['sideEffects']["bnpvPeriod"]
    total_declarations = script2["props"]["pageProps"]["cis"]["substances"][0]['sideEffects']["declarations"]["total"]

    decl_hom = script2["props"]["pageProps"]["cis"]["substances"][0]['sideEffects']["repartitionPerGender"]["male"]
    decl_fem = script2["props"]["pageProps"]["cis"]["substances"][0]['sideEffects']["repartitionPerGender"]["female"]

    # Les données démographiques au DataFrame
    new_row = {
        'Nom_medicament': Nom_medicament,
        'Estimation': Estimation,
        'Hommes (%)': pat_trait_hom['valuePercent'],
        'Femmes (%)': pat_trait_fem['valuePercent'],
        '20-59 (%)': age_20_59['valuePercent'],
        '60+ (%)': age_60_p['valuePercent'],
        'Période_min': periode_effects['minYear'],
        'Période_max': periode_effects['maxYear'],
        'Total_declarations': total_declarations,
        'Hommes_decl (%)': decl_hom['valuePercent'],
        'Femmes_decl (%)': decl_fem['valuePercent']
    }

    df_medications = pd.concat([df_medications, pd.DataFrame([new_row])], ignore_index=True)

    # Les effets indésirables au DataFrame
    side_effects = script2["props"]["pageProps"]["cis"]["substances"][0]['sideEffects']['repartitionPerPathology']
    for effect in side_effects:
        new_effect = {
            'Nom_medicament': Nom_medicament,
            'Pathologie': effect['range'],
            'Valeur': effect['value']
        }
        df_effects = pd.concat([df_effects, pd.DataFrame([new_effect])], ignore_index=True)

# Fichiers CSV
df_medications.to_csv('medications.csv', index=False)
df_effects.to_csv('effects.csv', index=False)

# Vérification
print(df_medications.head())
print(df_effects.head())


                                      Nom_medicament  \
0                  Aricept 10 mg, comprimé pelliculé   
1  Galantamine biogaran lp 16 mg, gélule à libéra...   
2       Memantine biogaran 20 mg, comprimé pelliculé   

                 Estimation  Hommes (%)  Femmes (%)  20-59 (%)  60+ (%)  \
0   Estimation : 4 000 / an        31.5        68.3        1.2     98.7   
1   Estimation : 3 000 / an        32.9        66.9        0.7     99.2   
2  Estimation : 10 000 / an        32.1        67.8        0.8     99.1   

  Période_min Période_max Total_declarations  Hommes_decl (%)  Femmes_decl (%)  
0        2014        2023                162             36.4             63.6  
1        2014        2023                129             44.2             55.8  
2        2014        2023                408             41.2             58.8  
                      Nom_medicament  \
0  Aricept 10 mg, comprimé pelliculé   
1  Aricept 10 mg, comprimé pelliculé   
2  Aricept 10 mg, comprimé pel

# Analyse des tables effects et medications

In [6]:
# Les données depuis les fichiers CSV
df_medications = pd.read_csv('/content/medications.csv')
df_effects = pd.read_csv('/content/effects.csv')

# Préparation des données pour les graphiques
def prepare_data():
    df_medications['Nom_du_médicament'] = df_medications['Nom_medicament'].str.split().str[0]
    df_sex_distribution = df_medications.groupby('Nom_du_médicament')[['Hommes (%)', 'Femmes (%)']].mean().reset_index()
    df_age_distribution = df_medications.groupby('Nom_du_médicament')[['20-59 (%)', '60+ (%)']].mean().reset_index()
    df_effects['Nom_du_médicament'] = df_effects['Nom_medicament'].str.split().str[0]
    df_effects_count = df_effects.groupby(['Nom_du_médicament', 'Pathologie'])['Valeur'].sum().reset_index()

    return df_sex_distribution, df_age_distribution, df_effects_count

df_sex_distribution, df_age_distribution, df_effects_count = prepare_data()


# NLP

In [7]:
# Les ressources nécessaires pour nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

# Chargement des données depuis les fichiers CSV
df_posts = pd.read_csv('/content/forum_posts_30_pages.csv')
df_medications = pd.read_csv('/content/medications.csv')
df_effects = pd.read_csv('/content/effects.csv')

# Les valeurs de la colonne 'Content' en chaînes de caractères
df_posts['Content'] = df_posts['Content'].astype(str)

# Les fonctions de nettoyage
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_newlines(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_digits(text):
    text = re.sub(r'\d', '', text)
    return text

def clean_text(text):
    text = text.lower()  # Convertir en minuscules
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    text = ' '.join([token for token in tokens if token not in stop_words])
    return text

stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Appliquation des fonctions de nettoyage aux posts du forum
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_URL(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_emoji(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_html(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_punct(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_newlines(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: remove_digits(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: clean_text(x))
df_posts['Content'] = df_posts['Content'].apply(lambda x: lemmatize_stemming(x))


big_text = ' '.join(df_posts['Content'].tolist())

# Liste des effets indésirables
effects_list = [
    "nausea", "vomiting", "headache", "dizziness", "fatigue",
    "abdominal pain", "diarrhea", "constipation", "loss of appetite",
    "muscle pain", "muscle spasms", "skin rash", "itching", "redness",
    "excessive sweating", "fever", "palpitations", "hypertension",
    "hypotension", "tachycardia", "bradycardia", "anemia", "weight loss",
    "weight gain", "swelling", "joint pain", "back pain", "chills",
    "dry mouth", "metallic taste", "increased thirst", "increased urination",
    "difficulty swallowing", "chest tightness", "tingling", "numbness",
    "lightheadedness", "malaise", "fatigue", "weakness", "tremors",
    "insomnia", "drowsiness", "confusion", "anxiety", "depression",
    "agitation", "tremors", "seizures", "memory loss", "balance disorders",
    "blurred vision", "restlessness", "panic attacks", "irritability",
    "hallucinations", "nightmares", "paranoia", "mood swings", "delirium",
    "dysarthria", "dystonia", "neuralgia", "paresthesia", "syncope",
    "stomach pain", "bloating", "acid reflux", "gastric ulcers",
    "indigestion", "flatulence", "belching", "heartburn", "intestinal gas",
    "bloody stools", "black stools", "jaundice", "liver dysfunction",
    "pancreatitis", "hepatitis", "colitis", "enteritis",
    "difficulty breathing", "shortness of breath", "cough", "nasal congestion",
    "wheezing", "sinusitis", "respiratory infection", "bronchospasm",
    "pneumonia", "dry throat", "hoarseness", "pharyngitis", "laryngitis",
    "asthma", "emphysema", "pleurisy",
    "hives", "dry skin", "hair loss", "eczema", "acne", "blisters",
    "photosensitivity", "skin discoloration", "bruising", "peeling skin",
    "dermatitis", "psoriasis", "alopecia", "urticaria",
    "chest pain", "heart attack", "stroke", "high blood pressure",
    "low blood pressure", "arrhythmia", "peripheral edema", "venous thrombosis",
    "heart palpitations", "heart failure", "angina", "myocarditis",
    "pericarditis", "varicose veins", "hypertension",
    "kidney pain", "urinary retention", "urinary incontinence", "kidney stones",
    "renal failure", "hematuria", "proteinuria", "dysuria",
    "impotence", "decreased libido", "menstrual irregularities",
    "breast tenderness", "gynecomastia", "hyperglycemia", "hypoglycemia",
    "electrolyte imbalance", "thyroid dysfunction", "increased cholesterol",
    "dehydration", "diabetes", "hyperthyroidism", "hypothyroidism",
    "taste changes", "visual disturbances", "hearing loss", "tinnitus",
    "anaphylaxis", "allergic reactions", "fat redistribution", "hepatotoxicity",
    "cardiotoxicity", "nephrotoxicity", "neurotoxicity", "ototoxicity",
    "musculoskeletal pain", "bone pain", "osteoporosis"
]

# Tokenisation
tokens = word_tokenize(big_text)

effect_terms_count = Counter([word for word in tokens if word in effects_list])

# Le nuage de mots avec tous les termes trouvés
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate_from_frequencies(effect_terms_count)
wordcloud.to_file("wordcloud.png")

# Analyseur de sentiments
sia = SentimentIntensityAnalyzer()

df_posts['sentiment'] = df_posts['Content'].apply(lambda x: sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0)

# Classification des sentiments
def classify_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df_posts['sentiment_class'] = df_posts['sentiment'].apply(classify_sentiment)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


# Dash

In [9]:
# Path to your images
wordcloud_path = "/content/wordcloud.png"
logo_path = "/content/logo.jpeg"

# Convert images to base64
encoded_wordcloud = base64.b64encode(open(wordcloud_path, 'rb').read()).decode('ascii')
encoded_logo = base64.b64encode(open(logo_path, 'rb').read()).decode('ascii')

# Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Mise en page de l'application
app.layout = dbc.Container([
    dbc.Navbar(
        dbc.Container([
            html.Img(src=f'data:image/jpeg;base64,{encoded_logo}', height="40px"),
            dbc.NavbarBrand(
                "Déclarations des effets indésirables associés aux médicaments de alzeimer",
                className="ms-2"
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
            dbc.Collapse(
                dbc.Nav(
                    [
                        dbc.NavItem(dbc.NavLink("Accueil", href="/")),
                        dbc.NavItem(dbc.NavLink("ANSM", href="/ansm")),
                        dbc.NavItem(dbc.NavLink("Forum Alzheimer", href="/forum")),
                    ],
                    className="ms-auto",
                    navbar=True
                ),
                id="navbar-collapse",
                navbar=True,
            ),
        ]),
        color="primary",
        dark=True,
    ),
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
], fluid=True)

# Page d'accueil
# Home Page
home_page = dbc.Container([
    html.Div([
        html.H3("Projet"),
        html.P("Le projet consiste en plusieurs étapes clés : collecte des données, analyse des sentiments, et visualisation des résultats. "
               "La collecte des données est effectuée par web scraping des forums spécialisés, suivi par une analyse de sentiments pour comprendre les perceptions des utilisateurs. "
               "Les résultats sont ensuite présentés sous forme de graphiques interactifs pour une interprétation facile."),
        html.Hr(),  # Add a horizontal rule for separation
        html.H3("Word Cloud des Effets Indésirables"),
        dbc.Card(
            dbc.CardBody(
                html.Div(
                    html.Img(src=f'data:image/png;base64,{encoded_wordcloud}', style={'width': '100%'}),
                    style={'display': 'flex', 'justify-content': 'center'}
                )
            ),
            className="mt-3"
        ),
        html.Hr(),  # Add a horizontal rule for separation
        html.H3("Design du Projet"),
        html.P("Le design du projet est axé sur la simplicité et l'accessibilité. Les données sont présentées de manière claire et concise pour garantir que "
               "les informations importantes sont facilement compréhensibles par tous les utilisateurs. Les graphiques interactifs permettent d'explorer les données "
               "en détail et de mieux comprendre les tendances et les sentiments associés aux médicaments.")
    ])
], fluid=True)

# Page ANSM
ansm_page = dbc.Container([
    html.H3("Sélectionner un Médicament"),
    dcc.Dropdown(
        id='medicament-dropdown',
        options=[{'label': nom, 'value': nom} for nom in df_sex_distribution['Nom_du_médicament'].unique()],
        value=df_sex_distribution['Nom_du_médicament'].unique()[0]  # Valeur par défaut
    ),
    dbc.Row([
        dbc.Col(dcc.Graph(id='sex-distribution'), width=6),
        dbc.Col(dcc.Graph(id='age-distribution'), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='effects-distribution'), width=12)
    ])
], fluid=True)

# Page Forum
forum_page = dbc.Container([
    dbc.Row([
        dbc.Col(dcc.Graph(id='sentiment-bar-chart'), width=6),
        dbc.Col(dcc.Graph(id='sentiment-box-plot'), width=6)
    ])
], fluid=True)

# Mise à jour du contenu en fonction de l'URL
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/ansm':
        return ansm_page
    elif pathname == '/forum':
        return forum_page
    else:
        return home_page

# Callbacks pour mettre à jour les graphiques de la page ANSM
@app.callback(
    Output('sex-distribution', 'figure'),
    Output('age-distribution', 'figure'),
    Output('effects-distribution', 'figure'),
    Input('medicament-dropdown', 'value')
)
def update_ansm_graphs(selected_medicament):
    filtered_sex_df = df_sex_distribution[df_sex_distribution['Nom_du_médicament'] == selected_medicament]
    filtered_age_df = df_age_distribution[df_age_distribution['Nom_du_médicament'] == selected_medicament]
    filtered_effects_df = df_effects_count[df_effects_count['Nom_du_médicament'] == selected_medicament]

    fig_sex = px.bar(filtered_sex_df, x='Nom_du_médicament', y=['Hommes (%)', 'Femmes (%)'],
                     title=f'Répartition des Patients par Sexe pour {selected_medicament}', barmode='group',
                     color_discrete_sequence=px.colors.qualitative.Pastel)

    age_data = filtered_age_df.melt(id_vars=["Nom_du_médicament"], value_vars=['20-59 (%)', '60+ (%)'],
                                    var_name="Tranche d'âge", value_name="Pourcentage")
    fig_age = px.pie(age_data, names='Tranche d\'âge', values='Pourcentage',
                     title=f'Répartition des Patients par Tranche d\'Âge pour {selected_medicament}',
                     color_discrete_sequence=px.colors.qualitative.Pastel)

    df_effects_sum = filtered_effects_df.groupby('Pathologie')['Valeur'].sum().reset_index()
    fig_effects = px.pie(df_effects_sum, values='Valeur', names='Pathologie',
                         title=f'Types d\'Effets Indésirables pour {selected_medicament}',
                         color_discrete_sequence=px.colors.qualitative.Pastel)

    return fig_sex, fig_age, fig_effects

# Analyseur de sentiments
sia = SentimentIntensityAnalyzer()
df_posts['sentiment'] = df_posts['Content'].apply(lambda x: sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0)

# Classification des sentiments
def classify_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df_posts['sentiment_class'] = df_posts['sentiment'].apply(classify_sentiment)

# Visualisation des sentiments
sentiment_counts = df_posts['sentiment_class'].value_counts().reset_index()
sentiment_counts.columns = ['sentiment_class', 'count']

# Callbacks pour mettre à jour les graphiques de la page Forum
@app.callback(
    Output('sentiment-bar-chart', 'figure'),
    Output('sentiment-box-plot', 'figure'),
    Input('url', 'pathname')
)
def update_forum_graphs(pathname):
    fig_bar = px.bar(sentiment_counts, x='sentiment_class', y='count',
                     title='Distribution des Sentiments dans les Posts du Forum',
                     color='sentiment_class', color_discrete_sequence=px.colors.qualitative.Pastel)

    fig_box = px.box(df_posts, x='sentiment_class', y='sentiment',
                     title='Scores de Sentiment par Classe de Sentiment',
                     color='sentiment_class', color_discrete_sequence=px.colors.qualitative.Pastel)

    return fig_bar, fig_box

# Exécutation l'application
nest_asyncio.apply()
app.run_server(mode='inline', height=1000)

<IPython.core.display.Javascript object>